## Transfer Learning for Fashion-MNIST
#### NOTE : Due to CPU and GPU limations of macbook i was not able to train as i started the amount of heat producd was very much for just quarter of first epoch and one epoch was taking like 6 to 7 min and there were 5 epoch still left i tried reducing the batch size but amount of heat produced was not reducing so to stop severe internal damage i did not continue process 
##### This code cell loads the data set from the database and then divides into training and validation set. After that it resizes the images into 224*224 and also converts the images from grayscale (1 channel) to RGB scale (3 channel)

In [4]:
import tensorflow as tf

(train_images, train_labels), (test_images, test_labels) = tf.keras.datasets.fashion_mnist.load_data()

train_images = tf.expand_dims(train_images/255.0, axis=-1)
test_images = tf.expand_dims(test_images/255.0, axis=-1)
print(train_images[0])

batch_size = 8

def pre_process(images, labels):
    images = tf.image.resize(images,[224,224])
    images = tf.image.grayscale_to_rgb(images)
    return images, labels

train_images, val_images = train_images[:50000], train_images[50000:60001]
train_labels, val_labels = train_labels[:50000], train_labels[50000:60001]

train_ds = tf.data.Dataset.from_tensor_slices((train_images, train_labels))
train_ds = train_ds.map(pre_process, num_parallel_calls=tf.data.AUTOTUNE)
train_ds = train_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

val_ds = tf.data.Dataset.from_tensor_slices((val_images, val_labels))
val_ds = val_ds.map(pre_process, num_parallel_calls=tf.data.AUTOTUNE)
val_ds = val_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)

test_ds = tf.data.Dataset.from_tensor_slices((test_images, test_labels))
test_ds = test_ds.map(pre_process, num_parallel_calls=tf.data.AUTOTUNE)
test_ds = test_ds.batch(batch_size).prefetch(tf.data.AUTOTUNE)


tf.Tensor(
[[[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]]

 [[0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.        ]
  [0.     

#### Creates a DNN with the help of keras functional API and and also loads the base ResNet50 model for the backbone

In [5]:
from tensorflow.keras.applications import ResNet50
from tensorflow.keras import layers,models

base_model = ResNet50(weights="imagenet",include_top=False,input_shape=(224,224,3))
base_model.trainable = False 

input = tf.keras.Input(shape=(224,224,3))
x = base_model(input,training = False)
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(128,activation='relu')(x)
x = layers.Dropout(0.5)(x)
output = layers.Dense(10,activation='softmax')(x)

model = models.Model(input,output)

#### Checks the availbility of the GPU

In [6]:
import tensorflow as tf

print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))
print("Devices:", tf.config.list_physical_devices())


Num GPUs Available:  1
Devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'), PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]


#### Compiles the model and trains it for the training and validation data

In [ ]:
metrics = [tf.keras.metrics.Accuracy(),
           tf.keras.metrics.Precision(),
           tf.keras.metrics.Recall(),
           tf.keras.metrics.F1Score()]

model.compile(loss="sparse_categorical_crossentropy", optimizer="adam",metrics=metrics)

tf.debugging.set_log_device_placement(True)

trained_model = model.fit(train_ds,validation_data=val_ds,epochs=5)

In [ ]:
trained_model.evaluate(test_ds)


